<a href="https://colab.research.google.com/github/TheNotoriousXxX/BusReservationSystem/blob/terminator3.0/paddle_converter_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ♻

!pip install -q paddlepaddle==2.5.0rc0 paddlenlp==2.5.2 ppdiffusers==0.14.1 diffusers==0.14.0 safetensors==0.3.1 huggingface-hub==0.13.4 transformers==4.27.0
!pip install -q omegaconf==2.3.0 fastcore==1.5.29 einops==0.6.1 gradio==3.19.1 numpy==1.22.4 lark==1.1.5 gdown==4.6.4 pytorch-lightning==2.0.2

import os, gdown, gc
import gradio as gr
from ppdiffusers import StableDiffusionPipeline
import paddle, torch
from safetensors.torch import save_file, load_file
from huggingface_hub import model_info, create_repo, create_branch, upload_folder
from huggingface_hub.utils import RepositoryNotFoundError, RevisionNotFoundError

def download_ckpt(ckpt_url, vae_url, is_safetensors):
    if is_safetensors:
        if "drive.google.com" in ckpt_url:
            gdown.download(url=ckpt_url, output="model.safetensors", quiet=False, fuzzy=True)
        else:
            os.system(f"wget {ckpt_url} -O model.safetensors")
    else:
        if "drive.google.com" in ckpt_url:
            gdown.download(url=ckpt_url, output="model.ckpt", quiet=False, fuzzy=True)
        else:
            os.system(f"wget {ckpt_url} -O model.ckpt")

    if vae_url != "":
        if "drive.google.com" in vae_url:
            gdown.download(url=vae_url, output="vae.ckpt", quiet=False, fuzzy=True)
        else:
            os.system(f"wget {vae_url} -O vae.ckpt")

        model = torch.load("model.ckpt", map_location="cpu")
        if "state_dict" in model:
          sd = model["state_dict"]
        else:
          sd = model
        full_model = False
        vae_model = torch.load("vae.ckpt", map_location="cpu")
        vae_sd = vae_model['state_dict']
        for vae_key in vae_sd:
          if vae_key.startswith("first_stage_model."):
            full_model = True
            break
        for vae_key in vae_sd:
          sd_key = vae_key
          if full_model:
            if not sd_key.startswith("first_stage_model."):
              continue
          else:
            if sd_key not in sd:
              sd_key = "first_stage_model." + sd_key
          if sd_key not in sd:
            continue
          sd[sd_key] = vae_sd[vae_key]
        torch.save(model, "model.ckpt")
        del model
        del vae_model
        del sd
        del vae_sd
        gc.collect()
    return "download ckpt done!"

def to_paddle(is_safetensors):
    if is_safetensors:
        paddle_pipe = StableDiffusionPipeline.from_pretrained_original_ckpt("model.safetensors")
        paddle_pipe.save_pretrained("paddle")
    else:
        paddle_pipe = StableDiffusionPipeline.from_pretrained_original_ckpt("model.ckpt")
        paddle_pipe.save_pretrained("paddle")
    del paddle_pipe
    gc.collect()
    return "convert to paddle done!"

def pt_to_paddle(pt_url):
    os.system("wget -q https://raw.githubusercontent.com/PaddlePaddle/PaddleNLP/v2.5.2/ppdiffusers/scripts/convert_diffusers_model/convert_diffusers_stable_diffusion_to_ppdiffusers.py -O convert_diffusers_stable_diffusion_to_ppdiffusers.py")
    os.system("sed -i 's/use_auth_token=True/use_auth_token=False/' convert_diffusers_stable_diffusion_to_ppdiffusers.py")
    os.system(f"python3 convert_diffusers_stable_diffusion_to_ppdiffusers.py --pretrained_model_name_or_path {pt_url} --output_path paddle")
    return "convert to paddle done!"

def push_paddle(model_to, token, branch):
    try:
        repo_exists = True
        r_info = model_info(model_to, token=token)
    except RepositoryNotFoundError:
        repo_exists = False
    finally:
        if repo_exists:
            print(r_info)
        else:
            create_repo(model_to, private=True, token=token)
    try:
        branch_exists = True
        b_info = model_info(model_to, revision=branch, token=token)
    except RevisionNotFoundError:
        branch_exists = False
    finally:
        if branch_exists:
            print(b_info)
        else:
            create_branch(model_to, branch=branch, token=token)
    upload_folder(folder_path="paddle", path_in_repo="", revision=branch, repo_id=model_to, commit_message=f"paddle - camenduru/converter", token=token)
    return "push pt done!"

def delete_paddle():
    os.system(f"rm -rf paddle *.yaml model.safetensors model.ckpt")
    return "delete paddle done!"

block = gr.Blocks()
with block:
    gr.Markdown(
    """
    ### ckpt pytorch to ppdiffusers paddle
    ckpt_url = <small>https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt or https://drive.google.com/file/d/file-id/view?usp=share_link or "https://civitai.com/api/download/models/5616?type=Model&format=PickleTensor"</small><br />
    paddle_model_to = camenduru/openjourney <br />
    branch = main <br />
    token = get from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens) new token role=write
    """)
    with gr.Group():
        with gr.Box():
            with gr.Row().style(equal_height=True):
                text_ckpt_url = gr.Textbox(show_label=False, max_lines=1, placeholder="ckpt_url")
                text_vae_url = gr.Textbox(show_label=False, max_lines=1, placeholder="vae_url")
                text_paddle_model_to = gr.Textbox(show_label=False, max_lines=1, placeholder="paddle_model_to")
                text_paddle_branch = gr.Textbox(show_label=False, value="main", max_lines=1, placeholder="paddle_branch")
                text_paddle_token = gr.Textbox(show_label=False, max_lines=1, placeholder="🤗 token")
                out_paddle = gr.Textbox(show_label=False)
            with gr.Row().style(equal_height=True):
                is_safetensors = gr.Checkbox(label="Is Safetensors", value=False)
                btn_download_ckpt = gr.Button("Download CKPT")
                btn_to_paddle = gr.Button("Convert to ppDiffusers Paddle")
                btn_push_paddle = gr.Button("Push ppDiffusers Paddle to 🤗")
                btn_delete_paddle = gr.Button("Delete ppDiffusers Paddle")
        btn_download_ckpt.click(download_ckpt, inputs=[text_ckpt_url, text_vae_url, is_safetensors], outputs=out_paddle)
        btn_to_paddle.click(to_paddle, inputs=[is_safetensors], outputs=out_paddle)
        btn_push_paddle.click(push_paddle, inputs=[text_paddle_model_to, text_paddle_token, text_paddle_branch], outputs=out_paddle)
        btn_delete_paddle.click(delete_paddle, outputs=out_paddle)
    gr.Markdown(
    """
    ### diffusers pytorch to ppdiffusers paddle <br />
    pt_model_from = dreamlike-art/dreamlike-diffusion-1.0 <br />
    paddle_model_to = camenduru/dreamlike-diffusion-1.0 <br />
    branch = paddle <br />
    token = get from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens) new token role=write <br />
    """)
    with gr.Group():
        with gr.Box():
            with gr.Row().style(equal_height=True):
                text_pt_model_from = gr.Textbox(show_label=False, max_lines=1, placeholder="pt_model_from")
                text_paddle_model_to = gr.Textbox(show_label=False, max_lines=1, placeholder="paddle_model_to")
                text_paddle_branch = gr.Textbox(show_label=False, value="main", max_lines=1, placeholder="paddle_branch")
                text_paddle_token = gr.Textbox(show_label=False, max_lines=1, placeholder="🤗 token")
                out_paddle = gr.Textbox(show_label=False)
            with gr.Row().style(equal_height=True):
                btn_to_paddle = gr.Button("Convert to ppDiffusers Paddle")
                btn_push_paddle = gr.Button("Push ppDiffusers Paddle to 🤗")
                btn_delete_paddle = gr.Button("Delete ppDiffusers Paddle")
        btn_to_paddle.click(pt_to_paddle, inputs=[text_pt_model_from], outputs=out_paddle)
        btn_push_paddle.click(push_paddle, inputs=[text_paddle_model_to, text_paddle_token, text_paddle_branch], outputs=out_paddle)
        btn_delete_paddle.click(delete_paddle, outputs=out_paddle)
block.launch(share=True, inline=False, debug=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.2/939.2 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 35.7 MB/s eta 0:00

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2ca33d219b70036aec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
